In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Sources

- https://medium.com/@manjabogicevic/multiple-linear-regression-using-python-b99754591ac0
- https://towardsdatascience.com/simple-and-multiple-linear-regression-in-python-c928425168f9

# Import du fichier CSV et traitement

Source de données : https://opendata.paris.fr/explore/dataset/les-arbres/export/?refine.libellefrancais=Epic%C3%A9a

In [26]:
df = (pd
          
        # Import
        .read_csv("data/epiceas.csv", sep=';')[['CIRCONFERENCEENCM', 'HAUTEUR (m)']]
        
        # Rename columns
        .rename(columns={
            'CIRCONFERENCEENCM': 'circ',
            'HAUTEUR (m)': 'height'
        })
      
        # Create and compute circ_sqrt column
        .assign(
            circ_sqrt=lambda x: np.sqrt(x['circ'])
        )
     
     )

df = df[['circ', 'circ_sqrt', 'height']]

df.head()

,circ,circ_sqrt,height
0,90.0,9.486833,12.0
1,120.0,10.954451,10.0
2,40.0,6.324555,4.0
3,105.0,10.246951,16.0
4,35.0,5.916080,7.0


# Nettoyage du jeu de données

In [27]:
df = df[df['circ'] > 0].copy()
df = df[df['height'] > 0].copy()
df = df[df['height'] < 30].copy()

# Préparation des données pour statsmodels

In [28]:
X = df.iloc[:, :-1].values
y = df.iloc[:, 2].values

In [40]:
# Add intercept
X = sm.add_constant(X)
# Equivalent to X = np.append(arr = np.ones((len(df), 1)).astype(int), values = X, axis = 1)

# Work on a copy
X_opt = X[:, [0,1,2]]

display(pd.DataFrame(X_opt).head())

,0,1,2
0,1.0,90.0,9.486833
1,1.0,120.0,10.954451
2,1.0,40.0,6.324555
3,1.0,105.0,10.246951
4,1.0,35.0,5.916080


# Calcul des p-valeurs

In [41]:
# Compute the linear regression
regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()
display(regressor_OLS.summary())

# Get the pvalues directly in an array
print("Pvalues:")
display(regressor_OLS.pvalues)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.538
Model:                            OLS   Adj. R-squared:                  0.531
Method:                 Least Squares   F-statistic:                     77.42
Date:                Mon, 25 Feb 2019   Prob (F-statistic):           5.03e-23
Time:                        16:23:30   Log-Likelihood:                -330.22
No. Observations:                 136   AIC:                             666.4
Df Residuals:                     133   BIC:                             675.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -6.3019      2.911     -2.165      0.032     -12.060      -0.544
x1            -0.0675      0.046     -1.467      0.145      -0.159       0.024
x2             2.4899      0.747      3.332      0.001       1.012       3.968
==============================================================================
Omnibus:                       11.933   Durbin-Watson:                   2.315
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               16.383
Skew:                           0.491   Prob(JB):                     0.000277
Kurtosis:                       4.389   Cond. No.                         945.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Pvalues:


array([0.03218757, 0.14474862, 0.00111646])

# Interprétation

Nous décidons de déterminer les variables à garder par "backward elimination" en se basant sur la p-value. Nous conserverons les variables avec une p-valeur supérieure à 0,05.

On constate que la p-valeur de x1 est élevée : 0,145. Ce paramètre n'est donc pas significatif, on l'élimine. Relançons la régression linéaire avec une seule variable (+ une constante).

# Nouvelle préparation des données

In [42]:
# Work on a copy
X_opt = X[:, [0,2]]

display(pd.DataFrame(X_opt).head())

,0,1
0,1.0,9.486833
1,1.0,10.954451
2,1.0,6.324555
3,1.0,10.246951
4,1.0,5.916080


# Nouveau calcul des p-valeurs

In [43]:
# Compute the linear regression
regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()
display(regressor_OLS.summary())

# Get the pvalues directly in an array
print("Pvalues:")
display(regressor_OLS.pvalues)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.530
Model:                            OLS   Adj. R-squared:                  0.527
Method:                 Least Squares   F-statistic:                     151.4
Date:                Mon, 25 Feb 2019   Prob (F-statistic):           9.41e-24
Time:                        16:27:54   Log-Likelihood:                -331.31
No. Observations:                 136   AIC:                             666.6
Df Residuals:                     134   BIC:                             672.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.2534      0.930     -2.422      0.017      -4.093      -0.414
x1             1.4065      0.114     12.304      0.000       1.180       1.633
==============================================================================
Omnibus:                       12.710   Durbin-Watson:                   2.321
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               17.510
Skew:                           0.521   Prob(JB):                     0.000158
Kurtosis:                       4.416   Cond. No.                         32.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Pvalues:


array([1.67558313e-02, 9.40909291e-24])

# Nouvelles interprétations

Au seuil de 5%, nous rejetons la nullité de la constante et de la variable "x1" (qui correspond à la racine carré de la circonférence). On conserve donc ces deux paramètres pour notre modèle.

Pour ce modèle, le coefficient de détermination R2 ajusté est de 0.527, ce qui n'est pas excellent mais reste correct puisqu'au dessus de la barre symbolique des 50%.